## Intoduction

All the ouput files from the condor jobs are kept in a local directory


Paths to these files are stored in `fileLists/isoDevV3.fls`

`Util.h` header is required for the isGoodTurnOn and getArea function


### Reading the Files

In [ ]:
import numpy as np
import os
import ROOT as rt

In [ ]:
rt.gInterpreter.ProcessLine('#include "Util.h"')

# Define the dataset Used

In [ ]:
RateStudyNtuple=' Single Neutrino Gun, 12X, re emulated with the latest 12_3_0_pre3, caloparams_2022_0_v1, Final recaliberated eT LUT'
EffStudyNtuple=' DYZToll , 12X, re emulated with the latest 12_3_0_pre3, caloparams_2022_0_v1,  Final recaliberated eT LUT'


DatsetInfoStr='Rates studied using \n\t:\t '+RateStudyNtuple+'\n'
DatsetInfoStr+='Efficiency studied using \n\t:\t '+EffStudyNtuple+'\n'

## Definie the Main data storege 

In [ ]:
def getOptionDescFromRateProgressionName(rateProgressionName):
    items=rateProgressionName.split('_')
    desc={}
    desc['option']=int(items[3].replace('option',''))
    desc['option_str']='_'.join(items[3:7])
    desc['optionParams']=(float(items[4].replace('p','.')),
                          float(items[5].replace('p','.')),
                          float(items[6].replace('p','.')))
    desc['EtThreshold'] = float(items[8].replace('p','.'))
    

    return desc
def getThresholdRate(rate):
    return  float( int((rate+0.1)*10)/10.0 )

#### Load the Et and Rate metrics

In [ ]:
def updateDataStore(data,RateForTurnons,AcceptanceForTurnons,TurnOnProgressionDir,RateProgressionDir):
    
    xAxisForNames = AcceptanceForTurnons.GetXaxis();
    nBins = AcceptanceForTurnons.GetNbinsX();
    AcceptanceMap={}
    for i in range(nBins):
        option_histName=xAxisForNames.GetBinLabel(i)
        if(option_histName==''):
            continue
        AcceptanceMap[option_histName]=AcceptanceForTurnons.GetBinContent(i)
        
    TurnOnMap={}    
    for ky in  TurnOnProgressionDir.GetListOfKeys():
        hName=ky.GetName()
        TurnOnMap[hName]=TurnOnProgressionDir.Get(hName)
    
    RateMap={}    
    for ky in  RateProgressionDir.GetListOfKeys():
        hName=ky.GetName()
        opt=int(hName.replace('rate_Progression','').split('_')[0])
        RateMap[opt]=RateProgressionDir.Get(hName)
        
    xAxisForNames = RateForTurnons.GetXaxis();
    nBins = RateForTurnons.GetNbinsX();
    for i in range(nBins):
        option_histName=xAxisForNames.GetBinLabel(i)
        if(option_histName==''):
            continue

        rate=RateForTurnons.GetBinContent(i)
        optionDesc=getOptionDescFromRateProgressionName(option_histName)

        opt=optionDesc['option']
        eT=optionDesc['EtThreshold']
        rateProgression=RateMap[opt]
        if opt not in data:
            data[opt]={}
            data[opt]['params']=optionDesc['optionParams']
            data[opt]['option_str']=optionDesc['option_str']
            data[opt]['rateProgression']=rateProgression

        acceptance=AcceptanceMap[option_histName]
        turnOn=TurnOnMap[option_histName]
        area=9.0
        area=rt.getIntegral(turnOn,eT,eT-7.0,eT+7.0,"",rt.nullptr,False)
        isGoodTurnOn=True
        
        if 'FixedEtMetrics' not in data[opt]:
            data[opt]['FixedEtMetrics']={}
        if eT not in data[opt]['FixedEtMetrics']:
                data[opt]['FixedEtMetrics'][eT]={}
        data[opt]['FixedEtMetrics'][eT]['rate']=rate
        data[opt]['FixedEtMetrics'][eT]['acceptance']=acceptance
        data[opt]['FixedEtMetrics'][eT]['turnOn']=turnOn
        data[opt]['FixedEtMetrics'][eT]['area']=area
        data[opt]['FixedEtMetrics'][eT]['isGoodTurnOn']=isGoodTurnOn

        if 'FixedRateMetrics' not in data[opt]:
            data[opt]['FixedRateMetrics']={}
        rate_rounded=getThresholdRate(rate)
       
        if rate_rounded in data[opt]['FixedRateMetrics']: 
            if (data[opt]['FixedRateMetrics'][rate_rounded]['eT'] < eT):
                pass
        else :
            data[opt]['FixedRateMetrics'][rate_rounded]={}
            data[opt]['FixedRateMetrics'][rate_rounded]['eT']=eT
            data[opt]['FixedRateMetrics'][rate_rounded]['acceptance']=acceptance
            data[opt]['FixedRateMetrics'][rate_rounded]['_rate']=rate
            data[opt]['FixedRateMetrics'][rate_rounded]['turnOn']=turnOn
            data[opt]['FixedRateMetrics'][rate_rounded]['area']=area
            data[opt]['FixedRateMetrics'][rate_rounded]['isGoodTurnOn']=isGoodTurnOn

    return data

### Run2 Turnon Store

Loads the Rate vs pt histogram from the root file. This histogram will be used to obtain the Et threshold for selcting the baseline turnon for comparisons ( when doing Fixed rate studies) . 

Loads All the availabe TurnsOns from Run2. Please customize the selection accodingly , so as to load the loose or tight turnons . The file loaded here is the output of the [turn_on_production.C](https://github.com/ats2008/EGTagAndProbe/blob/CMSSW_12_X/EGTagAndProbe/test/fitter/turn_on_production.C)

In [ ]:
fileRate.ls()

In [ ]:
fileRate=rt.TFile('run2RateHistograms/HistgramFile_EGRate_Run3MC_Recalib_inclusive_Run2Iso.root')

Run2RateHist=fileRate.Get('SingleEG_rate_LooseIso')

fileRun2TurnOns=rt.TFile('run3MC_12_0_2_RecalibV2_turnon.root')
Run2TurnOnMap={}
Run2AcceptanceMap={}
PassOptions={}
FailOptions={}
for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if not hName.startswith('hist_L1Et'):
        continue
    if 'loose' not in hName:
        continue
    if 'def' in hName:
        continue
    et=int(hName.split('_')[1].replace('L1Et',''))
    if 'Pass' in hName:
        PassOptions[et]=fileRun2TurnOns.Get(hName)
    elif 'Fail' in hName:
        FailOptions[et]=fileRun2TurnOns.Get(hName)
    else:
        continue

for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'Graph_from_hist_L1Et' not in hName:
        continue
    if 'loose' not in hName:
        continue
    if 'def' in hName:
        continue
    eT=int(hName.split('_')[3].replace('L1Et','').replace('Efficiency',''))
    Run2TurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    acceptance=-1.0
    if eT not in PassOptions or eT not in FailOptions:
        print("\t\t Problem !! eT not found in pass and fain maps !! eT = ",eT)
    else :
        acceptance=PassOptions[eT].Integral()*1.0/(PassOptions[eT].Integral() + FailOptions[eT].Integral())
    Run2AcceptanceMap[eT]=acceptance
    print('For ',eT,' Adding ',hName, "Acceptance = ",acceptance)

#### Load the Files and data

In [ ]:
fList=open('fileLists/isoDevV3p2.fls')
txt=fList.readlines()
fileNames=[]
for l in txt:
    fileNames.append(l[:-1])
fList.close()

In [ ]:
data={}
TFILE_STORE={}
count=0
for fName in fileNames:
    count+=1
    print(count," / ",len(fName)," Adding File ",fName)
    file=rt.TFile(fName)
    RateForTurnons  = file.Get('RateForTurnons')
    EtForTurnons    = file.Get('EtForTurnons')
    AcceptanceForTurnons = file.Get('AcceptanceForTurnons')
    TurnOnProgressionDir = file.Get('turn_on_progression')
    RateProgressionDir = file.Get('rate_progression')
    data=updateDataStore(data,RateForTurnons,AcceptanceForTurnons,TurnOnProgressionDir,RateProgressionDir)
    TFILE_STORE[fName]=file
#     if count>2:
#         break
print()
print("Loaded ",count," Files")
print("Loaded ",len(data)," options")

### Testing the root macro functions

In [ ]:
# data.keys()

In [ ]:
opt=354
opt2=888

eT=30

cvs=rt.TCanvas("cvs")
cvs.cd()
data[opt]['FixedEtMetrics'][eT]['turnOn'].Draw()
data[opt2]['FixedEtMetrics'][eT]['turnOn'].Draw("same")
cvs.Draw()

#isGoodTurnOn

t1=data[opt]['FixedEtMetrics'][eT]['turnOn']
t2=data[opt2]['FixedEtMetrics'][eT]['turnOn']

y=rt.isGoodTurnON(t2,t1,10,19)
print(y)

### Integral

print(data[opt]['FixedEtMetrics'][eT])
turnOn=data[opt]['FixedEtMetrics'][eT]['turnOn']

area=rt.getIntegral(turnOn,eT,eT-4.0,eT+4.0,"pref",rt.nullptr,False)
print("area = ",area)

## Get Baseline and Best Option for Fixed Rates

In [ ]:
def getBaselineForFixedRate(fRate,Run2RateHist,Run2TurnOnMap,Run2AcceptanceMap):
    nBins=Run2RateHist.GetNbinsX()
    eT=-1
    idx=-1
    rtFixHere=-1e6
    for i in range(nBins):
        rate=Run2RateHist.GetBinContent(i)
        if(rate > fRate ):
            continue
        else:
            if (fRate - rate ) < (fRate - rtFixHere):
                rtFixHere=rate
                idx=i    
    if idx>=0:
        eT=int(Run2RateHist.GetBinCenter(idx))
    if eT < 0:
        print(" No thresold found with 'rate < Fixed Rate ' (",fRate,"kHz in run 2 ")
        return (rt.nullptr,-1,-1,-1)
    if eT not in Run2TurnOnMap:
        print(" No Turnons found for thresold eT = ",eT," in run2 turn on collection")
        return (rt.nullptr,-1,-1,-1)
    
    turnon=Run2TurnOnMap[eT]
    nBins=turnon.GetN()
    idxA=-10
    idxB=-10
    print("N in turnons ",nBins)
    for i in range(nBins):
        eff=turnon.GetPointY(i)
        if(eff > 0.67 and idxA < 0):
            idxA=i-1
        if(eff > 0.91 and idxB < 0):
            idxB=i
    if(idxB <0  or idxA <0 ):
        print("Something wrong with the idx finding for fRate ",fRate," which gave eT ",eT)
    print("\t Baseline found to be at eT = ",eT," for fixed Rate ",fRate,
              " indexes : ",idxA,"(","{0:0.3f}".format(turnon.GetPointY(idxA)),")",
          idxB,"(","{0:0.3f}".format(turnon.GetPointY(idxB)),")" )
    return (Run2TurnOnMap[eT],idxA,idxB,eT,Run2AcceptanceMap[eT])
    
    
# dataForValidation={'baseLine':[],'p1':[],'p2':[],'rslt':[],'turnOn':[]}
def getAllOptionsDataForFixedRate(data,F_Rate,Baseline=None):
    dataForFixedRate={'option':[],
                      'option_pars':[],
                      'eT':[],'rate':[],
                      'area':[],
                      'acceptance':[],
                      'turnOn':[],
                      'isGoodTurnOn':[],
                      'rateProgression':[]}
    for opt in data:
        idx=-1
        rateFixHere=-1e6
        eTmaxSearched=-1.0
        rateForETmaxSearched=-1.0
        for rate in data[opt]['FixedRateMetrics']:
            if eTmaxSearched < data[opt]['FixedRateMetrics'][rate]['eT']:
                eTmaxSearched=data[opt]['FixedRateMetrics'][rate]['eT']
                rateForETmaxSearched=rate
            if rate > F_Rate:
                continue
            else:
                if (F_Rate - rate ) < (F_Rate - rateFixHere):
                    rateFixHere=rate
        if(rateFixHere < 0.0 ):
            print("\tNo Et thresolds could be reached for option ",opt," at fixed Rate",F_Rate,
                  "( rate : ",rateForETmaxSearched," for eT ",eTmaxSearched," )" )
            continue
        vals=data[opt]['FixedRateMetrics'][rateFixHere]
        
        isGoodTurnON=False #
        if Baseline[0]:
            isGoodTurnON=rt.isGoodTurnON(Baseline[0],vals['turnOn'],Baseline[1],Baseline[2])
#             dataForValidation['baseLine'].append(Baseline[0])
#             dataForValidation['p1'].append(Baseline[1])
#             dataForValidation['p2'].append(Baseline[2])
#             dataForValidation['turnOn'].append(vals['turnOn'])
#             dataForValidation['rslt'].append(isGoodTurnON)
#             print("Got isGoodTurnOns as ",isGoodTurnON)
            
        dataForFixedRate['option'].append(opt)
        dataForFixedRate['eT'].append(vals['eT'])
        dataForFixedRate['rate'].append(vals['_rate'])
        dataForFixedRate['area'].append(vals['area'])
        dataForFixedRate['isGoodTurnOn'].append(isGoodTurnON)
        dataForFixedRate['acceptance'].append(vals['acceptance'])
        dataForFixedRate['turnOn'].append(vals['turnOn'])
        dataForFixedRate['rateProgression'].append(data[opt]['rateProgression'])
        dataForFixedRate['option_pars'].append(data[opt]['params'])
    return dataForFixedRate

## Scanning the fixed rates

In [ ]:
fixedRates=[10,11,13,14,15,16,16.7,17,18,19,20]

In [ ]:
dataForFixedRates={}
for fR in fixedRates:
    print("Doing for Fixed Rate : ",fR)
    Baseline=getBaselineForFixedRate(fR,Run2RateHist,Run2TurnOnMap,Run2AcceptanceMap)
    dataForFixedRates[fR]={}
    dataForFixedRates[fR]['Baseline']=Baseline
    dataForFixedRates[fR]['scan']=getAllOptionsDataForFixedRate(data,fR,Baseline)

In [ ]:
# n=0
# t1=dataForValidation['baseLine'][n]
# t2=dataForValidation['turnOn'][n]
# p1=dataForValidation['p1'][n]
# p2=dataForValidation['p2'][n]
# rslt=dataForValidation['rslt'][n]
# print(p1,p2)
# y=rt.isGoodTurnON(t1,t2,p1,p2)
# print(y,rslt)

# cvs=rt.TCanvas("cvs")
# cvs.cd()
# t1.Draw()
# t2.Draw("same")
# cvs.Draw()

## Export the results

In [ ]:
prefixBase="results/isoDevV3/SingleEGLoose/FixedRateScans"
metricStr='area'

if not os.path.exists(prefixBase):
    print("making : ",prefixBase)
    os.system('mkdir -p '+prefixBase)
else:
    print("removing : ",prefixBase)
    os.system('rm -r '+prefixBase)
    print("making : ",prefixBase)
    os.system('mkdir -p '+prefixBase)
summaryFileName=prefixBase+'/'+'summaryForFixedRateScans.txt'
summaryFile=open(summaryFileName,'w')
summaryFile.write(DatsetInfoStr)
for fRate in dataForFixedRates:
    summaryFile.write('='*20+'\n')
    summaryFile.write("Summary at Fixed Rate = "+str(fRate)+" kHz : \n \n")
    
    prefix=prefixBase+'/'+str(fRate)+'kHz'
    print("For Fixed rate ",fRate," Saving results to ",prefix)
    
    
    if not os.path.exists(prefix):
        print("making : ",prefix)
        os.system('mkdir -p '+prefix)

    #################   Baseline Details to the summary ###########
    
    summaryFile.write("  Baseline details the fixed rate \n")
    BaselineTurnOn=dataForFixedRates[fRate]['Baseline'][0]
    eTBaseLine=dataForFixedRates[fRate]['Baseline'][3]
    areaBaseline=rt.getIntegral(BaselineTurnOn,eTBaseLine,eTBaseLine-7.0,eTBaseLine+7.0,prefix+'/Baseline_',rt.nullptr,True)
    acceptanceBaseline=dataForFixedRates[fRate]['Baseline'][4]
    baselineString ='\tBaseline eT   : '+str(eTBaseLine) 
    baselineString+='\tBaseline Area : '+str(areaBaseline)
    baselineString+='\tBaseline Acceptance   : '+str(acceptanceBaseline)
    summaryFile.write(baselineString)
    summaryFile.write("\n\n")
    #####################################################################
    
    fixedRateData=dataForFixedRates[fRate]['scan']
    metric=np.asarray(fixedRateData[metricStr])
    sortedIdx=np.argsort(-1*metric)
    isGoodTurnONMask=np.asarray(fixedRateData['isGoodTurnOn'])==True 

    allOptResults=open(prefix+'/allOptionsScan.txt','w')
    allOptResults.write(DatsetInfoStr+'\n')
    allOptResults.write("Full options scans at fixed Rate "+str(fRate)+' kHz\n')
    allOptResults.write("In decending order of the metric "+metricStr+'\n')
    allOptResults.write(baselineString+'\n')
    
    allOptResultsWithSelection=open(prefix+'/selectedOptions.txt','w')
    allOptResultsWithSelection.write("Full options scans at fixed Rate "+str(fRate)+' kHz\n')
    allOptResultsWithSelection.write("In decending order of the metric "+metricStr+'\n')
    allOptResultsWithSelection.write(baselineString+'\n')
    
    keysToPrint=['option','option_pars','eT','rate','isGoodTurnOn','area','acceptance']
    strToW=''
    for j in range(len(keysToPrint)):
        ky=keysToPrint[j]
        strToW+=ky
        strToW+="\t"
    strToW+='\n'
    allOptResults.write(strToW)
    allOptResultsWithSelection.write(strToW)
    
    eTsInTheScan=np.unique(np.asarray(fixedRateData['eT'])[isGoodTurnONMask]  ) 
    topOptionIdxForEachEt={i:[] for i in eTsInTheScan}
    count=0
    for i in sortedIdx:
        strToW=''
        for j in range(len(keysToPrint)):
            ky=keysToPrint[j]
            strToW+=str(fixedRateData[ky][i])
            strToW+="\t"
        strToW+='\n'
        allOptResults.write(strToW)
        
        if not isGoodTurnONMask[i]:
            continue;
        count+=1
        eT=fixedRateData['eT'][i]
        topOptionIdxForEachEt[eT].append(i)
        strToW=''
        for j in range(len(keysToPrint)):
            ky=keysToPrint[j]
            strToW+=str(fixedRateData[ky][i])
            strToW+="\t"
        strToW+='\n'
        allOptResultsWithSelection.write(strToW)
    allOptResults.close()
    allOptResultsWithSelection.close()
    print("  Number of options passing the section " ,count)
    summaryFile.write("\n  Number of options passing the selection : "+str(count)+"\n\n")
    

    
    #################   TOP n Options   to Summary ################
    
    summaryFile.write("  Best options for the fixed rate \n")
    count=0
    strToW='\t'
    for j in range(len(keysToPrint)):
        strToW+=keysToPrint[j]+'\t'
    summaryFile.write(strToW+'\n')
    
    for i in sortedIdx:
        if not isGoodTurnONMask[i]:
            continue;
        count+=1
    
        area=rt.getIntegral(fixedRateData['turnOn'][i],eT,eT-7.0,eT+7.0,prefix+'/',BaselineTurnOn,True)
        if count > 5 :
            break
        strToW='\t'
        for j in range(len(keysToPrint)):
            strToW+=str(fixedRateData[keysToPrint[j]][i])+"\t"
        summaryFile.write(strToW+'\n')
    summaryFile.write("\n")
    #################   TOP n Options for unique eT  to Summary ################
    
    for eT in eTsInTheScan:
        summaryFile.write("  Best options for eT : "+str(eT)+'\n')
        count=0
        strToW='\t'
        for j in range(len(keysToPrint)):
            strToW+=keysToPrint[j]+'\t'
        summaryFile.write(strToW+'\n')

        for i in topOptionIdxForEachEt[eT]:
    #         if not isGoodTurnONMask[i]:
    #             continue;
            count+=1
            area=rt.getIntegral(fixedRateData['turnOn'][i],eT,eT-7.0,eT+7.0,prefix+'/',BaselineTurnOn,True)
            if count > 3 :
                break
            strToW='\t'
            for j in range(len(keysToPrint)):
                strToW+=str(fixedRateData[keysToPrint[j]][i])+"\t"
            summaryFile.write(strToW+'\n')
        summaryFile.write("\n")
    summaryFile.write("\n")
        
summaryFile.close() 
print('\nsummary written into ',summaryFileName)